In [ ]:
import sys
import os

if 'google.colab' in sys.modules:
  print("You are in colab")
  !pip install pyngrok drake
  from google.colab import drive
  drive.mount('/content/drive')
  os.chdir('/content/drive/MyDrive/gcs-motion-planning-main/jupyter_notebooks')

import numpy as np

from pyngrok import ngrok

# Drake Libraries
from pydrake.geometry import ( StartMeshcat )
from pydrake.geometry.optimization import (SaveIrisRegionsYamlFile)
from pydrake.systems.framework import DiagramBuilder
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.visualization import AddDefaultVisualization

# add parent folders to path
sys.path.append(os.path.abspath('..'))

# Update the import
from src.manipulation import running_as_notebook
from src.auxiliar.auxiliar_functions import (VisualizeConnectivity)
from src.auxiliar.homemade_gcs import GcsTrajOpt
from src.auxiliar.iris_options import create_iris_options
from src.auxiliar.joint_teleop import JointTeleop
from src.auxiliar.helper import VisualizeRegions
from src.auxiliar.auxiliar_functions import LoadRobot

In [ ]:
# Start the visualizer (only run this once).
meshcat = StartMeshcat()
if 'google.colab' in sys.modules:
  # Create a tunnel to the Meshcat server
  ngrok.set_auth_token("2rdws4UBYTqZ1NiiyEbJcWRnByV_7D9Jg8K3YVyZzbFMN1ysg")
  public_url = ngrok.connect(7000, "http")
  print(f"Meshcat is accessible at: {public_url}")

In [ ]:
import src.auxiliar.importing_objects as io

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
LoadRobot(plant)
plant.Finalize()
AddDefaultVisualization(builder, meshcat)
diagram = builder.Build()

In [ ]:
diagram_context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(diagram_context)
print("Diagram context: ",diagram_context)
print("Plant context: ", plant_context)

In [ ]:
q_initial = plant.GetPositions(plant_context)
print("q_initial: ", q_initial)

In [ ]:
diagram.ForcedPublish(diagram_context)